In [2]:
# https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
import torch
import torch
import torch.nn as nn
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import random
from sklearn.preprocessing import MinMaxScaler

In [3]:
# model

class Model(nn.Module):
    
    def __init__(self, input_size=1, hidden_size=100, output_size=1, num_layers=1):
        
        super(Model, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.linear = nn.Linear(hidden_size, output_size)
        self.hidden_cell = (torch.zeros(1,1,self.hidden_size),
                            torch.zeros(1,1,self.hidden_size))
    
    #  methods 
    def forward(self, x):
        
        out, self.hidden_cell = self.lstm(x)
        out = self.linear(out.view(len(x), -1))
        
        return out[-1]

In [4]:
df_train = pd.DataFrame({'Act1':[x + random()*10 for x in range(0, 100)],
                         'Act2':50+np.sin(np.linspace(0, 2*np.pi, 100))*50})
df_test = pd.DataFrame({'Act1':[x + random()*10 for x in range(101, 201)],
                         'Act2':50+np.sin(np.linspace(0, 2*np.pi, 100))*50})

In [5]:
df_train.tail()

,Act1,Act2
95,98.815588,37.442601
96,103.650758,40.537438
97,97.065560,43.670377
98,107.994370,46.828804
99,104.077217,50.000000


In [6]:
df_test.tail()

,Act1,Act2
95,198.332727,37.442601
96,197.497137,40.537438
97,207.042002,43.670377
98,202.707042,46.828804
99,205.076423,50.000000


In [7]:
series_1 = list(df_train['Act1'])
series_2 = list(df_train['Act2'])
#series_test = list(df_test['Act1'])

In [8]:
print(series_1)

[6.506076388404988, 9.657566572109603, 5.224270386092508, 7.944070454227157, 8.908403246385202, 5.214193966606563, 6.625476395274407, 14.317928375754153, 17.760294297162538, 16.609727782075904, 16.324265305751045, 11.162544522648755, 21.812194226118795, 14.07459837805602, 18.399163426336465, 22.009203931282368, 17.3921810802159, 20.692157134642457, 22.64361054160458, 19.698713119486964, 20.183733026458047, 25.637621391124416, 27.96316566510025, 24.589899128520194, 29.29164470815651, 34.793231256284265, 33.728419519816754, 32.87277525291067, 28.404648284148248, 33.80431162599995, 32.8462503248597, 32.07843177731224, 35.62987247668023, 37.62232771917084, 37.67091626300109, 41.47248090158353, 45.46469962373809, 46.27547519884051, 46.55489441597808, 43.87927147481591, 48.357394670888475, 43.326286275653686, 43.30369078534612, 43.92704331394688, 44.594595787051425, 48.84280428441015, 53.38507570262764, 50.86728543826147, 49.99511829661038, 49.761365281421675, 59.36417281219846, 55.209403994

In [9]:
train_inout_seq = []
train_window = 10
L = len(series_1)

for i in range(L-train_window):
    train_seq = []
    for j in range(train_window):
        train_seq.append([series_1[i+j:i+j+1][0],
                          series_2[i+j:i+j+1][0]])
    train_seq = torch.tensor(train_seq)
        
    train_label = torch.tensor([series_1[i+train_window:i+train_window+1]])
    train_inout_seq.append((train_seq ,train_label))

In [10]:
train_inout_seq

[(tensor([[ 6.5061, 50.0000],
          [ 9.6576, 53.1712],
          [ 5.2243, 56.3296],
          [ 7.9441, 59.4626],
          [ 8.9084, 62.5574],
          [ 5.2142, 65.6017],
          [ 6.6255, 68.5831],
          [14.3179, 71.4897],
          [17.7603, 74.3098],
          [16.6097, 77.0320]]),
  tensor([[16.3243]])),
 (tensor([[ 9.6576, 53.1712],
          [ 5.2243, 56.3296],
          [ 7.9441, 59.4626],
          [ 8.9084, 62.5574],
          [ 5.2142, 65.6017],
          [ 6.6255, 68.5831],
          [14.3179, 71.4897],
          [17.7603, 74.3098],
          [16.6097, 77.0320],
          [16.3243, 79.6454]]),
  tensor([[11.1625]])),
 (tensor([[ 5.2243, 56.3296],
          [ 7.9441, 59.4626],
          [ 8.9084, 62.5574],
          [ 5.2142, 65.6017],
          [ 6.6255, 68.5831],
          [14.3179, 71.4897],
          [17.7603, 74.3098],
          [16.6097, 77.0320],
          [16.3243, 79.6454],
          [11.1625, 82.1394]]),
  tensor([[21.8122]])),
 (tensor([[ 7.9441, 59

In [11]:
# example
input_size=2
hidden_size= 100 
output_size=1
num_layers=1

model = Model(input_size, hidden_size, output_size, num_layers)

In [12]:
epochs = 150
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_size),
                             torch.zeros(1, 1, model.hidden_size))
 
        y_pred = model(seq)
 
        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
 
    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')
 
print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

d:\10_python\python-3.8.2\python\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch:   1 loss: 6929.83447266
epoch:  26 loss: 651.56817627
epoch:  51 loss: 241.73568726
epoch:  76 loss: 28.67761421
epoch: 101 loss: 51.79019928
epoch: 126 loss: 9.93553925
epoch: 149 loss: 6.2632060051


In [14]:
list(model.parameters())

[Parameter containing:
 tensor([[ 1.6061e-01, -2.2572e-01],
         [ 2.0789e-01,  1.3144e-01],
         [ 9.9350e-02, -6.9967e-02],
         [-1.2168e-02,  1.6701e-01],
         [ 8.5018e-02, -2.7729e-01],
         [-5.2326e-02,  8.8820e-02],
         [ 7.5610e-02,  1.5956e-01],
         [ 1.4581e-01,  1.9494e-01],
         [-1.7851e-01,  1.1245e-01],
         [ 1.4600e-01, -1.0811e-01],
         [ 1.5173e-01, -1.8049e-01],
         [ 1.4148e-01,  3.1891e-02],
         [-7.3629e-02,  4.5788e-01],
         [ 3.7979e-01, -1.0482e-01],
         [ 5.7758e-02,  7.5676e-02],
         [ 1.9159e-01,  1.9326e-01],
         [-8.1035e-02, -8.3672e-02],
         [ 1.3736e-01,  8.0613e-02],
         [ 8.8774e-02, -2.4894e-01],
         [-1.9232e-01,  1.2176e-01],
         [ 1.1576e-01, -2.0738e-01],
         [-8.9597e-02, -4.5619e-02],
         [ 9.6111e-04,  3.9473e-02],
         [ 1.3075e-01,  2.0574e-02],
         [ 2.8399e-01,  2.2435e-01],
         [ 1.0753e-01,  7.6657e-02],
         [ 2.20

In [15]:
model.parameters

<bound method Module.parameters of Model(
  (lstm): LSTM(2, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)>